I/we certify that the code and data in this assignment were generated independently, using only the tools and resources defined in the course and that I/we did not receive any external help, coaching or contributions during the production of this work.

# Part 1

In [1]:
!apt-get install -y libgl1-mesa-dev libgl1-mesa-glx libglew-dev libosmesa6-dev software-properties-common
!apt-get install -y patchelf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1-mesa-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
libgl1-mesa-dev set to manually installed.
software-properties-common is already the newest version (0.96.24.32.18).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
Suggested packages:
  glew-utils
The following NEW packages will be installed:
  libgl1-mesa-glx libglew-dev libglew2.0 libosmesa6 libosmesa6-dev
0 upgraded, 5 newly installed, 0 to remove and 42 not upgraded.
Need to get 2,916 kB of archives.
After this operation, 12.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libgl1-mesa-glx amd64 20.0.8-0ubuntu1~18.04.1 [5,532 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libglew2.0 amd64 2.0.0-5 [140 kB]
Get:3 http://archive.

In [2]:
!pip install free-mujoco-py
!pip install Box2D
!pip install box2d-py
!pip install gym[all]
!pip install gym[Box_2D]

     |████████████████████████████████| 14.1 MB 3.1 MB/s 
     |████████████████████████████████| 203 kB 55.1 MB/s 
     |████████████████████████████████| 3.4 MB 24.0 MB/s 
     |████████████████████████████████| 4.3 MB 42.7 MB/s 
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: imageio
    Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 1.3 MB 5.1 MB/s 
     |████████████████████████████████| 448 kB 5.1 MB/s 
     |████████████████████████████████| 120 kB 5.0 MB/s 
  ERROR: Failed building wheel for mujoco-py
  Running setup.py clean for mujoco-py
Failed to build mujoco-py
    Running setup.py install for mujoco-py ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-ixydq_w5/mujoco-py_41f2d126ed5a4990a9bf2df20eddcc76/setup.py'"'"'; __file__='"'"'/tmp/pip-install-ixydq_w5/mujoco-py_41f2d126ed5a4990a9bf2df20eddcc76/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-r39csmaw/install-record.txt --single-version-externally-manag

In [3]:
import gym, os
import mujoco_py
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum
from itertools import count
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical

Compiling /usr/local/lib/python3.7/dist-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /usr/local/lib/python3.7/dist-packages/mujoco_py/cymj.pyx
running build_ext
building 'mujoco_py.cymj' extension
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7/usr
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7/usr/local
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7/usr/local/lib
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuexten

In [4]:
class GridEnvironment(gym.Env):
    metadata = { 'render.modes': [] }
    
    def __init__(self, env_type):
        self.observation_space = gym.spaces.Discrete(20)
        self.action_space = gym.spaces.Discrete(4)
        self.max_timesteps = 15
        self.prob = 0.95 if env_type == 'Stochastic' else 1
        self.actions = Enum('Actions', 'UP RIGHT DOWN LEFT')
        
    def reset(self):
        self.timestep = 0
        self.reward = 0
        self.agent_pos = [0, 0]
        self.goal_pos = [3, 4]
        self.gold_1 = [0, 3]
        self.gold_2 = [1, 2]
        self.gold_3 = [2, 1]
        self.gold_4 = [2, 3]
        self.gold_5 = [3, 0]
        self.state = np.zeros((4, 5))
        self.state[tuple(self.agent_pos)] = 0.75
        self.state[tuple(self.goal_pos)] = 0.25
        self.state[tuple(self.gold_1)] = 1
        self.state[tuple(self.gold_2)] = 1
        self.state[tuple(self.gold_3)] = 0.5
        self.state[tuple(self.gold_4)] = 1
        self.state[tuple(self.gold_5)] = 1
        observation = self.state.flatten()
        return observation
    
    def step(self, action):
        if random.random() < self.prob:
          if action == self.actions['UP'].value:
            self.agent_pos[0] -= 1
          if action == self.actions['RIGHT'].value:
            self.agent_pos[1] += 1
          if action == self.actions['DOWN'].value:
            self.agent_pos[0] += 1
          if action == self.actions['LEFT'].value:
            self.agent_pos[1] -= 1
          
        self.agent_pos[0] = np.clip(self.agent_pos[0], 0, 3)
        self.agent_pos[1] = np.clip(self.agent_pos[1], 0, 4)
        self.state = np.zeros((4, 5))
        self.state[tuple(self.agent_pos)] = 0.75
        self.state[tuple(self.goal_pos)] = 0.25
        self.state[tuple(self.gold_1)] = 1
        self.state[tuple(self.gold_2)] = 1
        self.state[tuple(self.gold_3)] = 0.5
        self.state[tuple(self.gold_4)] = 1
        self.state[tuple(self.gold_5)] = 1
        observation = self.state.flatten()
        
        if(self.agent_pos == self.goal_pos):
          self.reward = 10
        elif(self.agent_pos == self.gold_1):
          self.reward = 1
        elif(self.agent_pos == self.gold_2):
          self.reward = 1
        elif(self.agent_pos == self.gold_3):
          self.reward = -5
        elif(self.agent_pos == self.gold_4):
          self.reward = 1
        elif(self.agent_pos == self.gold_5):
          self.reward = 1
        else:
          self.reward = 0
          
        self.timestep += 1
        done = (self.timestep >= self.max_timesteps) or (self.agent_pos == self.goal_pos)
        info = {}
        
        return observation, self.reward, done, info
        
    def render(self):
        plt.imshow(self.state)

In [5]:
class Actor(nn.Module):
  def __init__(self, state_size, action_size):
    super(Actor, self).__init__()
    self.net = nn.Sequential(nn.Linear(state_size, 128), nn.ReLU(), nn.Linear(128, 256), nn.ReLU(), nn.Linear(256, action_size))
  def forward(self, state):
    output = self.net(state)
    return Categorical(F.softmax(output, dim=-1))

In [6]:
class Critic(nn.Module):
  def __init__(self, state_size, action_size):
    super(Critic, self).__init__()
    self.net = nn.Sequential(nn.Linear(state_size, 128), nn.ReLU(), nn.Linear(128, 256), nn.ReLU(), nn.Linear(256, 1))
  def forward(self, state):
    return self.net(state)

In [7]:
class ActorCritic:
  def __init__(self, env):
    self.env = env
    self.rewards_per_episode = []
    self.rewards_eval_per_episode = []
  def reset(self):
    self.state = env.reset()
    self.probs = []
    self.values = []
    self.rewards = []
    self.masks = []
    self.entropy = 0
  def training(self, actor, critic, episodes, learning_rate=0.01):
    opt_A = optim.Adam(actor.parameters())
    opt_C = optim.Adam(critic.parameters(), lr=learning_rate)
    for episode in range(episodes):
      self.reset()
      for i in count():
        self.state = torch.FloatTensor(self.state)
        dist, value = actor(self.state), critic(self.state)
        action = dist.sample()
        self.state, reward, done, info = env.step(action.cpu().numpy())
        p = dist.log_prob(action).unsqueeze(0)
        self.entropy += dist.entropy().mean()
        self.probs.append(p)
        self.values.append(value)
        self.rewards.append(torch.tensor([reward], dtype=torch.float))
        self.masks.append(torch.tensor([1-done], dtype=torch.float))
        if done:
          if(episode % 100 == 0 or episode == (episodes - 1)):
            print('Episode: {}, Steps: {}, Reward: {}'.format(episode, i, int(sum(self.rewards))))
          self.rewards_per_episode.append(int(sum(self.rewards)))
          break
      next_value = critic(torch.FloatTensor(self.state))
      self.probs = torch.cat(self.probs)
      self.values = torch.cat(self.values)
      advantage = torch.cat(self.compute(next_value)).detach() - self.values
      actor_loss = -(self.probs * advantage.detach()).mean()
      critic_loss = advantage.pow(2).mean()
      opt_A.zero_grad()
      opt_C.zero_grad()
      actor_loss.backward()
      critic_loss.backward()
      opt_A.step()
      opt_C.step()
    env.close()
  def compute(self, next, gamma=0.99):
    vals = []
    i=len(self.rewards)-1
    while i!=-1:
      next = self.rewards[i] + gamma * next * self.masks[i]
      vals.insert(0, next)
      i-=1
    return vals
  def evaluation(self, actor, critic, episodes):
    for episode in range(episodes):
      self.reset()
      for i in count():
        self.state = torch.FloatTensor(self.state)
        dist, value = actor(self.state), critic(self.state)
        action = dist.sample()
        self.state, reward, done, info = env.step(action.cpu().numpy())
        p = dist.log_prob(action).unsqueeze(0)
        self.entropy += dist.entropy().mean()
        self.probs.append(p)
        self.values.append(value)
        self.rewards.append(torch.tensor([reward], dtype=torch.float))
        self.masks.append(torch.tensor([1-done], dtype=torch.float))
        if done:
          self.rewards_eval_per_episode.append(int(sum(self.rewards)))
          break

In [8]:
env = GridEnvironment("")
actor = Actor(env.observation_space.n, env.action_space.n)
critic = Critic(env.observation_space.n, env.action_space.n)
ac = ActorCritic(env)
ac.training(actor, critic, episodes=1000)
ac.evaluation(actor, critic, episodes=10)

Episode: 0, Steps: 11, Reward: 11
Episode: 100, Steps: 7, Reward: 5
Episode: 200, Steps: 8, Reward: 11
Episode: 300, Steps: 10, Reward: 11
Episode: 400, Steps: 10, Reward: 12
Episode: 500, Steps: 11, Reward: 12
Episode: 600, Steps: 6, Reward: 12
Episode: 700, Steps: 10, Reward: 14
Episode: 800, Steps: 6, Reward: 12
Episode: 900, Steps: 8, Reward: 13
Episode: 999, Steps: 13, Reward: 17


In [ ]:
plt.figure(1)
plt.title('Training')
plt.xlabel('Episodes')
plt.ylabel('Reward per Episode')
plt.plot(ac.rewards_per_episode)
plt.figure(2)
plt.title('Evaluation')
plt.xlabel('Episodes')
plt.ylabel('Reward per Episode')
plt.plot(ac.rewards_eval_per_episode)
plt.show()

# Part 2

## Environment 1(Lunar Lander)

In [ ]:
class Actor(nn.Module):
  def __init__(self, state_size, action_size):
    super(Actor, self).__init__()
    self.net = nn.Sequential(nn.Linear(state_size, 128), nn.ReLU(), nn.Linear(128, 256), nn.ReLU(), nn.Linear(256, action_size))
  def forward(self, state):
    output = self.net(state)
    return Categorical(F.softmax(output, dim=-1))

In [ ]:
class Critic(nn.Module):
  def __init__(self, state_size, action_size):
    super(Critic, self).__init__()
    self.net = nn.Sequential(nn.Linear(state_size, 128), nn.ReLU(), nn.Linear(128, 256), nn.ReLU(), nn.Linear(256, 1))
  def forward(self, state):
    return self.net(state)

In [ ]:
env = gym.make('LunarLander-v2')
env.seed(1234)
actor = Actor(env.observation_space.shape[0], env.action_space.n)
critic = Critic(env.observation_space.shape[0], env.action_space.n)
ac = ActorCritic(env)
ac.training(actor, critic, episodes=1000, learning_rate=0.01)
ac.evaluation(actor, critic, episodes=10)

In [ ]:
plt.figure(1)
plt.title('Training')
plt.xlabel('Episodes')
plt.ylabel('Reward per Episode')
plt.plot(ac.rewards_per_episode)
plt.figure(2)
plt.title('Evaluation')
plt.xlabel('Episodes')
plt.ylabel('Reward per Episode')
plt.plot(ac.rewards_eval_per_episode)
plt.show()

## Environment 2(OpenAI Ant)

In [ ]:
class Actor(nn.Module):
  def __init__(self, state_size, action_size):
    super(Actor, self).__init__()
    self.net = nn.Sequential(nn.Linear(state_size, 128), nn.ReLU(), nn.Linear(128, 256), nn.ReLU(), nn.Linear(256, action_size))
  def forward(self, state):
    output = self.net(state)
    return Categorical(F.softmax(output, dim=-1))

In [ ]:
class Critic(nn.Module):
  def __init__(self, state_size, action_size):
    super(Critic, self).__init__()
    self.net = nn.Sequential(nn.Linear(state_size, 128), nn.ReLU(), nn.Linear(128, 256), nn.ReLU(), nn.Linear(256, 1))
  def forward(self, state):
    return self.net(state)

In [ ]:
env = gym.make('Ant-v2')
env.seed(1234)
actor = Actor(env.observation_space.shape[0], env.action_space.shape[0])
critic = Critic(env.observation_space.shape[0], env.action_space.shape[0])
ac = ActorCritic(env)
ac.training(actor, critic, episodes=200, learning_rate=0.01)
ac.evaluation(actor, critic, episodes=10)

Episode: 0, Steps: 999, Reward: -7777
Episode: 100, Steps: 999, Reward: 863
Episode: 199, Steps: 999, Reward: 1001


In [ ]:
plt.figure(1)
plt.title('Training')
plt.xlabel('Episodes')
plt.ylabel('Reward per Episode')
plt.plot(ac.rewards_per_episode)
plt.figure(2)
plt.title('Evaluation')
plt.xlabel('Episodes')
plt.ylabel('Reward per Episode')
plt.plot(ac.rewards_eval_per_episode)
plt.show()